## **1D FDTD Solver with ABC Boundary Condition**

Python adaptation of John B. Schneider's C programs from chapter 6 of his textbook *Understanding the Finite-Difference Time-Domain
Method.*

Chapter 6 of Schneider's book is located [HERE](https://eecs.wsu.edu/~schneidj/ufdtd/chap6.pdf) and his GitHub source code is available [HERE](https://github.com/john-b-schneider/uFDTD).

### Animation Setup

Functions to create animations of the 1D FDTD solvers.

In [1]:
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from IPython import display


def ani_init():
    """Clear the axes for the animation."""
    ax1.cla()
    ax2.cla()
    ax1.set_xlim(0, SIZE)
    ax1.set_ylim(-1, 1)
    ax2.set_ylim(-1 / 377, 1 / 377)
    ax1.set_xlabel("Node Number")
    ax1.set_ylabel("E-Field", color="b")
    ax2.set_ylabel("H-Field", color="r")
    ax2.yaxis.set_label_position("right")
    ax2.yaxis.tick_right()
    plt.tight_layout()  # Needed to prevent right label being cut off


def animate(*args):
    """Draw the E-field and H-field magnitudes along X-axis at current time step."""
    ez, hy = next(sim_step)
    ax1.cla()
    ax2.cla()
    ax1.plot(x, ez, "-b")
    ax2.plot(x[: len(hy)], hy, "--r")
    ax1.set_xlim(0, SIZE)
    ax1.set_ylim(-1, 1)
    ax2.set_ylim(-1 / 377, 1 / 377)
    ax1.set_xlabel("Node Number")
    ax1.set_ylabel("E-Field", color="b")
    ax2.set_ylabel("H-Field", color="r")
    ax2.yaxis.set_label_position("right")
    ax2.yaxis.tick_right()
    if len(args) == 2:
        ax1.axvspan(100, SIZE, facecolor="green", alpha=0.3)
    elif len(args) > 2:
        ax1.axvspan(100, args[2], facecolor="green", alpha=0.3)
        ax1.axvspan(args[2], SIZE, facecolor="blue", alpha=0.3)
    plt.tight_layout()  # Needed to prevent right label being cut off


def html5_video(fargs=False):
    """Jupyter notebook must have animation converted to HTML5 video to display."""
    if fargs:
        mpl_animation = animation.FuncAnimation(
            fig, animate, fargs=fargs, frames=maxTime, init_func=ani_init, interval=25
        )
    else:
        mpl_animation = animation.FuncAnimation(
            fig, animate, frames=maxTime, init_func=ani_init, interval=25
        )
    html_video = mpl_animation.to_html5_video()
    html_display = display.HTML(html_video)
    display.display(html_display)
    plt.close("all")  # Ensure plots aren't left open and consuming memory

### Program 6.1

**gridhalfspace.c:** Function to initialize the Grid such that there are two halfspaces: free space to the left and a dielectric with ǫr = 9 to the right.

In [2]:
import numpy as np

SIZE = 200
maxTime = 500

class Grid:
    def __init__(self):
        self.EPSR = None
        self.imp0 = None       # Impedance of free space
        self.SizeX = None      # Size of domain
        self.MaxTime = None    # Duration of simulation
        self.Cdtds = None      # Courant number
        self.Time = None       # Current time step
        
        self.Ez = None
        self.Ceze = None
        self.Cezh = None
        self.Hy = None
        self.Chyh = None
        self.Chye = None


def gridInit(g):
    g.EPSR = 9
    g.imp0 = 377         # Impedance of free space
    g.SizeX = SIZE       # Size of domain
    g.MaxTime = maxTime  # Duration of simulation
    g.Cdtds = 1          # Courant number
    g.Time = 0

    g.Ez = np.zeros(g.SizeX)
    g.Ceze = np.ones(g.SizeX)
    g.Cezh = np.ones(g.SizeX) * g.imp0
    g.Cezh[100:] /= g.EPSR
    g.Hy = np.zeros(g.SizeX-1)
    g.Chyh = np.ones(g.SizeX-1)
    g.Chye = np.ones(g.SizeX-1) / g.imp0

### Program 6.2

**abcdemo1.c:** One-dimensional FDTD simulation employing a first-order ABC.

In [3]:
def abcdemo1():
    g = Grid()
    
    gridInit(g)
    abcInit(g)
    tfsfInit(g)
    snapshotInit(g)

    # Do time stepping
    for g.Time in range(g.MaxTime):
        updateH3(g)   # Update magnetic field
        tfsfUpdate(g)  # Correct field on TFSF boundary
        updateE3(g)   # Update Electric field
        abc(g)        # Apply ABC after E-field update
        #snapshot(g)   # Take a snapshot
        yield g.Ez, g.Hy
    

### Program 6.3
**abcfirst.c:** Implementation of a first-order absorbing boundary condition.

In [4]:
import sys

initDone = 0
ezOldLeft = 0
ezOldRight = 0
abcCoefLeft = 0
abcCoefRight = 0


def abcInit(g):
    global initDone, abcCoefLeft, abcCoefRight
    initDone = 1

    # Calculate coefficient on left end of grid
    temp = np.sqrt(g.Cezh[0] * g.Chye[0])
    abcCoefLeft = (temp - 1) / (temp + 1)

    # Calculate coefficient on right end of grid
    temp = np.sqrt(g.Cezh[g.SizeX - 1] * g.Chye[g.SizeX - 2])
    abcCoefRight = (temp-1) / (temp+1)


def abc(g):
    global initDone, ezOldLeft, abcCoefLeft, ezOldRight, abcCoefRight

    # Check if abcInit() has been called
    if not initDone:
        print('abc: abcInit must be called before abc.')
        sys.exit(-1)
        
    # ABC for left side of grid
    g.Ez[0] = ezOldLeft + abcCoefLeft * (g.Ez[1] - g.Ez[0])
    ezOldLeft = g.Ez[1]
    
    # ABC for right side of grid
    g.Ez[g.SizeX - 1] = ezOldRight + abcCoefRight * (g.Ez[g.SizeX - 2] - g.Ez[g.SizeX - 1])
    ezOldRight = g.Ez[g.SizeX - 2]

### Other required functions

Files associated with ABC demo as outlined in Figure 6.2.

**tfsf.c:** Initialize and update total field/scattered field excitation.

In [5]:
tfsfBoundary = 0


def tfsfInit(g):
    global tfsfBoundary
    tfsfBoundary = 50
    ezIncInit(g)  # initialize source function


def tfsfUpdate(g):
    # check if tfsfInit() has been called
    if tfsfBoundary <= 0:
        print("tfsfUpdate: tfsfInit must be called before tfsfUpdate.\n"
              "            Boundary location must be set to positive value.\n")
        sys.exit(-1)

    # correct Hy adjacent to TFSF boundary
    g.Hy[tfsfBoundary] -= ezInc(g.Time, 0.0)*g.Chye[tfsfBoundary]

    # correct Ez adjacent to TFSF boundary */
    g.Ez[tfsfBoundary+1] += ezInc(g.Time+0.5, -0.5);

**ezinc3.c:** Functions to calculate the source function (i.e., the incident field).

In [6]:
delay = 0
width = 0
cdtds = 0


def ezIncInit(g):
    global delay, width, cdtds
    cdtds = g.Cdtds
    delay = 30
    width = 10


def ezInc(time, location):
    """Calculate source function at given time and location."""
    if width <= 0:
        print("ezInc: must call ezIncInit before ezInc.\n"
              "       Width must be positive.\n")
        sys.exit(-1)

    return np.exp(-((time - delay - location / cdtds) / width)**2)

**update3.c:** Functions to update the electric and magnetic fields.

In [7]:
def updateH3(g):
    """Update magnetic field."""
    for mm in range(g.SizeX - 1):
        g.Hy[mm] = g.Chyh[mm] * g.Hy[mm] + \
            g.Chye[mm] * (g.Ez[mm + 1] - g.Ez[mm])


def updateE3(g):
    """Update electric field."""
    for mm in range(1, g.SizeX - 1):
        g.Ez[mm] = g.Ceze[mm] * g.Ez[mm] + \
            g.Cezh[mm] * (g.Hy[mm] - g.Hy[mm - 1])

**snapshot.c:** Function to take a snapshot of a 1D grid.

In [8]:
def snapshotInit(g):
    pass


def snapshot(g):
    pass

### Run ABC Demo

In [9]:
sim_step = abcdemo1()

x = np.arange(SIZE)
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
html5_video((True,))

### Program 6.4

**abcsecond.c:** The abcInit() and abc() functions for implementation of a second-order ABC.

In [10]:
initDone = 0
ezOldLeft1, ezOldLeft2 = 0, 0
ezOldRight1, ezOldRight2 = 0, 0
abcCoefLeft, abcCoefRight = 0, 0


def abcInit2(g):
    """Initizalization function for second-order ABC."""
    global ezOldLeft1, ezOldLeft2
    global ezOldRight1, ezOldRight2
    global abcCoefLeft, abcCoefRight
    global initDone
    initDone = 1

    ezOldLeft1 = np.zeros(3)
    ezOldLeft2 = np.zeros(3)
    ezOldRight1 = np.zeros(3)
    ezOldRight2 = np.zeros(3)

    abcCoefLeft = np.zeros(3)
    abcCoefRight = np.zeros(3)

    # calculate coefficients on left end of grid
    temp1 = np.sqrt(g.Cezh[0] * g.Chye[0])
    temp2 = 1.0 / temp1 + 2.0 + temp1
    abcCoefLeft[0] = -(1.0 / temp1 - 2.0 + temp1) / temp2
    abcCoefLeft[1] = -2.0 * (temp1 - 1.0 / temp1) / temp2
    abcCoefLeft[2] = 4.0 * (temp1 + 1.0 / temp1) / temp2

    # calculate coefficients on right end of grid
    temp1 = np.sqrt(g.Cezh[g.SizeX - 1] * g.Chye[g.SizeX - 2])
    temp2 = 1.0 / temp1 + 2.0 + temp1
    abcCoefRight[0] = -(1.0 / temp1 - 2.0 + temp1) / temp2
    abcCoefRight[1] = -2.0 * (temp1 - 1.0 / temp1) / temp2
    abcCoefRight[2] = 4.0 * (temp1 + 1.0 / temp1) / temp2


def abc2(g):
    """Second-order ABC."""

    # check if abcInit() has been called
    if not initDone:
        print("abc: abcInit must be called before abc.\n")
        sys.exit(-1)

    # ABC for left side of grid
    g.Ez[0] = abcCoefLeft[0] * (g.Ez[2] + ezOldLeft2[0]) \
        + abcCoefLeft[1] * (ezOldLeft1[0] + ezOldLeft1[2] -
                            g.Ez[1] - ezOldLeft2[1]) \
        + abcCoefLeft[2] * ezOldLeft1[1] - ezOldLeft2[2]

    # ABC for right side of grid */
    g.Ez[g.SizeX-1] = abcCoefRight[0] * (g.Ez[g.SizeX - 3] + ezOldRight2[0]) \
        + abcCoefRight[1] * (ezOldRight1[0] + ezOldRight1[2] -
                             g.Ez[g.SizeX - 2] - ezOldRight2[1]) \
        + abcCoefRight[2] * ezOldRight1[1] - ezOldRight2[2]

    # update stored fields
    for mm in range(3):
        ezOldLeft2[mm] = ezOldLeft1[mm]
        ezOldLeft1[mm] = g.Ez[mm]

        ezOldRight2[mm] = ezOldRight1[mm]
        ezOldRight1[mm] = g.Ez[g.SizeX - 1 - mm]

## Run ABC Demo 2

Second-order ABC.

**abcdemo2.c:**

In [11]:
def abcdemo2():
    g = Grid()
    
    gridInit(g)
    abcInit2(g)
    tfsfInit(g)
    snapshotInit(g)

    # Do time stepping
    for g.Time in range(g.MaxTime):
        updateH3(g)    # Update magnetic field
        tfsfUpdate(g)  # Correct field on TFSF boundary
        updateE3(g)    # Update Electric field
        abc2(g)        # Apply ABC after E-field update
        #snapshot(g)   # Take a snapshot
        yield g.Ez, g.Hy
    

In [12]:
sim_step = abcdemo2()

x = np.arange(SIZE)
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
html5_video((True,))